In [ ]:
import pandas as pd
from functools import reduce
from google.oauth2 import service_account

Extração dos dados

In [ ]:
atletas = pd.read_excel(f'C:/Users/Yann/Desktop/Estudo/etl_excel/data/Athletes.xlsx')
treinadores = pd.read_excel(f'C:/Users/Yann/Desktop/Estudo/etl_excel/data/Coaches.xlsx')
etradas_generos = pd.read_excel(f'C:/Users/Yann/Desktop/Estudo/etl_excel/data/EntriesGender.xlsx')
medalhas = pd.read_excel(f'C:/Users/Yann/Desktop/Estudo/etl_excel/data/Medals.xlsx')
times = pd.read_excel(f'C:/Users/Yann/Desktop/Estudo/etl_excel/data/Teams.xlsx')

Transformação

In [ ]:
# Atletas por modalidade e país

atletas_disc = atletas.groupby(['NOC', 'Discipline']) ['Name'].nunique()
atletas_disc = pd.DataFrame(atletas_disc).reset_index(drop=False).rename(columns={'Name': 'numero_atletas',
                                                                   'Discipline': 'modalidade',
                                                                   'NOC': 'pais'})



In [ ]:
# Treinadores por modalidade e país

treinadores_disc = treinadores.groupby(['NOC', 'Discipline']) ['Name'].nunique()
treinadores_disc = pd.DataFrame(treinadores_disc).reset_index(drop=False).rename(columns={'Name': 'numero_treinadores',
                                                                                          'Discipline': 'modalidade',
                                                                                          'NOC': 'pais'})
treinadores_disc

In [ ]:
# Times por gênero

times['Categoria'] = times.Event.apply(lambda x: 'Masculino' if 'Men' in x
                                        else ('Feminino' if 'Women' in x
                                            else ('Misto' if 'Mixed' in x
                                                else 'Outros')))
                                            

In [ ]:
times.Categoria.value_counts()

In [ ]:

times_masc = pd.DataFrame(times[times.Categoria == 'Masculino'].groupby(['NOC', 'Discipline'])['Name'].nunique()).\
reset_index().rename(columns={'Name': 'times_masculinos',
                              'Discipline': 'modalidade',
                              'NOC': 'pais'})

times_fem = pd.DataFrame(times[times.Categoria == 'Feminino'].groupby(['NOC', 'Discipline']) ['Name'].nunique()).\
reset_index().rename(columns={'Name': 'times_femininos',
                              'Discipline': 'modalidade',
                              'NOC': 'pais'})

times_mistos = pd.DataFrame(times[times.Categoria == 'Misto'].groupby(['NOC', 'Discipline']) ['Name'].nunique()).\
reset_index().rename(columns={'Name': 'times_mistos',
                              'Discipline': 'modalidade',
                              'NOC': 'pais'})

times_outros = pd.DataFrame(times[times.Categoria == 'Outros'].groupby(['NOC', 'Discipline']) ['Name'].nunique()).\
reset_index().rename(columns={'Name': 'outros',
                              'Discipline': 'modalidade',
                              'NOC': 'pais'})


Juntando as tabelas

In [ ]:
#Utilizando a biblioteca reduce para unir todos os df's de uma só vez

dfs = [atletas_disc,
            treinadores_disc,
            times_masc,
            times_fem,
            times_mistos,
            times_outros]

df_final = reduce(lambda left, right: pd.merge(left,
                                               right, on=['pais', 'modalidade'],
                                               how='outer'),
                                               dfs).fillna(0)

# Converte colunas com números para int

cols = ['numero_atletas', 
        'numero_treinadores', 
        'times_masculinos', 
        'times_femininos', 
        'times_mistos', 
        'outros']

df_final[cols] = df_final[cols].astype(int)



Carregamento para o Big Query

In [ ]:
credentials = service_account.Credentials.from_service_account_file(filename='C:/Users/Yann/Desktop/Estudo/etl_excel/GBQ.json', 
                                                                    scopes=["https://www.googleapis.com/auth/cloud-platform"])

df_final.to_gbq(credentials=credentials, 
                destination_table='curso_etl.etl_excel', 
                if_exists='replace')